## https://www.dataquest.io/blog/sql-basics/

### Covering
 - SELECT
 - WHERE
 - GROUP BY
 - JOIN
 - Conditionals
 - Create DB & Table
 - INSERT, UPDATE, DELETE

In [1]:
import os
os.getcwd()
# os.chdir("/Users/stevewatkins/python3/demos/sqlite") # relative path: scripts dir is under Users/stevewatkins

'C:\\Users\\stevewatkins\\AnacondaProjects'

## create conn. and "read results" pd function

In [2]:
import sqlite3
import pandas as pd

db = sqlite3.connect("hubway.db")

def run_query(query):
    return pd.read_sql_query(query,db)

## read metadata about sqlite DB tables

In [3]:
query = """
SELECT * 
FROM sqlite_master
ORDER BY name;
"""
run_query(query)

,type,name,tbl_name,rootpage,sql
0,table,MyTable,MyTable,33344,CREATE TABLE MyTable(rid integer)
1,table,stations,stations,33340,"CREATE TABLE stations (id INTEGER, station TEX..."
2,table,trips,trips,2,"CREATE TABLE trips (id INTEGER, duration INTEG..."


## select all from table

In [4]:
query = "SELECT * FROM trips LIMIT 5;"
run_query(query)

,id,duration,start_date,start_station,end_date,end_station,bike_number,sub_type,zip_code,birth_date,gender
0,1,9,2011-07-28 10:12:00,23,2011-07-28 10:12:00,23,B00468,Registered,'97217,1976.0,Male
1,2,220,2011-07-28 10:21:00,23,2011-07-28 10:25:00,23,B00554,Registered,'02215,1966.0,Male
2,3,56,2011-07-28 10:33:00,23,2011-07-28 10:34:00,23,B00456,Registered,'02108,1943.0,Male
3,4,64,2011-07-28 10:35:00,23,2011-07-28 10:36:00,23,B00554,Registered,'02116,1981.0,Female
4,5,12,2011-07-28 10:37:00,23,2011-07-28 10:37:00,23,B00554,Registered,'97214,1983.0,Female


## select 2 columns from table with custom order

In [5]:
query = """
SELECT 
     duration
    ,start_date 
FROM trips 
ORDER BY duration DESC ,start_date ASC
LIMIT 5;
"""
run_query(query)

,duration,start_date
0,9999,2012-09-26 18:34:00
1,9998,2011-08-09 10:42:00
2,9998,2012-08-15 12:11:00
3,9997,2011-10-09 02:30:00
4,9996,2011-11-19 13:48:00


## select all where duration is max, using limit & order desc

In [6]:
query = '''
SELECT duration ,*
FROM trips
ORDER BY duration DESC
LIMIT 1;
'''

run_query(query)

,duration,id,duration,start_date,start_station,end_date,end_station,bike_number,sub_type,zip_code,birth_date,gender
0,9999,541247,9999,2012-09-26 18:34:00,54,2012-09-26 21:21:00,54,T01078,Casual,,,


## select max duration per each start_station, using group by

In [7]:
query = """
SELECT max(duration) ,avg(duration),start_station 
FROM trips
group by start_station
order by 1 desc
limit 5;

"""

run_query(query)

,max(duration),avg(duration),start_station
0,9999,1051.836841,54
1,9998,866.412956,40
2,9997,1326.620569,60
3,9996,801.142841,4
4,9996,770.682636,35


## where clause to filter resultset

In [8]:
query = '''
SELECT * 
FROM trips
WHERE duration > 9990
--limit 5
;
'''

run_query(query)

,id,duration,start_date,start_station,end_date,end_station,bike_number,sub_type,zip_code,birth_date,gender
0,4768,9994,2011-08-03 17:16:00,22,2011-08-03 20:03:00,24,B00002,Casual,,,
1,8448,9991,2011-08-06 13:02:00,52,2011-08-06 15:48:00,24,B00174,Casual,,,
2,11341,9998,2011-08-09 10:42:00,40,2011-08-09 13:29:00,42,B00513,Casual,,,
3,24455,9995,2011-08-20 12:20:00,52,2011-08-20 15:07:00,17,B00552,Casual,,,
4,55771,9994,2011-09-14 15:44:00,40,2011-09-14 18:30:00,40,B00139,Casual,,,
5,81191,9993,2011-10-03 11:30:00,22,2011-10-03 14:16:00,36,B00474,Casual,,,
6,89335,9997,2011-10-09 02:30:00,60,2011-10-09 05:17:00,45,B00047,Casual,,,
7,124500,9992,2011-11-09 09:08:00,22,2011-11-09 11:55:00,40,B00387,Casual,,,
8,133967,9996,2011-11-19 13:48:00,4,2011-11-19 16:35:00,58,B00238,Casual,,,
9,147451,9996,2012-03-23 14:48:00,35,2012-03-23 17:35:00,33,B00550,Casual,,,


# AND condition

In [9]:
query = '''
SELECT * 
FROM trips
WHERE (duration >= 9990) AND (sub_type = "Registered")
ORDER BY duration DESC;
'''

run_query(query)

,id,duration,start_date,start_station,end_date,end_station,bike_number,sub_type,zip_code,birth_date,gender
0,315737,9995,2012-07-03 18:28:00,12,2012-07-03 21:15:00,12,B00250,Registered,'02120,1964.0,Male


# Count with AND condition

In [10]:
query = '''
SELECT 
  COUNT(*) AS "Total Trips by Registered Users"  
FROM trips
WHERE sub_type = "Registered";
'''

run_query(query)

,Total Trips by Registered Users
0,1105192


# Aggregating data with GROUP BY

In [11]:
query = '''
SELECT 
   sub_type
  ,COUNT(*) AS "Total Trips by Registered Users"  
FROM trips
group by sub_type
order by 2 desc;
'''

run_query(query)

,sub_type,Total Trips by Registered Users
0,Registered,1105192
1,Casual,464809


In [12]:
query = '''
SELECT sub_type, AVG(duration) AS "Average Duration"
FROM trips
GROUP BY sub_type;
'''

run_query(query)

,sub_type,Average Duration
0,Casual,1519.643897
1,Registered,657.026067


# which bike was used for the most trips?

In [13]:
query = '''
SELECT bike_number as "Bike Number", COUNT(*) AS "Number of Trips"
FROM trips
GROUP BY bike_number
ORDER BY COUNT(*) DESC
LIMIT 1;
'''

run_query(query)

,Bike Number,Number of Trips
0,B00490,2120


# Arithmetic Operators

In [14]:
query = '''
SELECT AVG(duration)
FROM trips
WHERE (2018 - birth_date) > 30;
'''

run_query(query)

,AVG(duration)
0,920.445299


In [15]:
query = '''
SELECT 
 birth_date  as "birth year"
,(2017 - birth_date) as "Years Old"
,date('now') as "now date"
,(strftime('%Y',date('now'))) as "now Year string"
,((strftime('%Y',date('now'))) - birth_date) as "Years Old revised"
FROM trips
WHERE (2018 - birth_date) > 30
and birth_date <> '';
'''

run_query(query)

,birth year,Years Old,now date,now Year string,Years Old revised
0,1976.0,41.0,2018-09-24,2018,42.0
1,1966.0,51.0,2018-09-24,2018,52.0
2,1943.0,74.0,2018-09-24,2018,75.0
3,1981.0,36.0,2018-09-24,2018,37.0
4,1983.0,34.0,2018-09-24,2018,35.0
5,1951.0,66.0,2018-09-24,2018,67.0
6,1971.0,46.0,2018-09-24,2018,47.0
7,1971.0,46.0,2018-09-24,2018,47.0
8,1983.0,34.0,2018-09-24,2018,35.0
9,1956.0,61.0,2018-09-24,2018,62.0


# Joins

In [16]:
query = '''
SELECT *
FROM stations
where id = 23
LIMIT 5;
'''
run_query(query)

,id,station,municipality,lat,lng
0,23,Mayor Thomas M. Menino - Government Center,Boston,42.359677000000005,-71.059364


In [17]:
query = '''
SELECT *
FROM trips
where start_station = 23 -- or end_station = 23
LIMIT 5;
'''
run_query(query)

,id,duration,start_date,start_station,end_date,end_station,bike_number,sub_type,zip_code,birth_date,gender
0,1,9,2011-07-28 10:12:00,23,2011-07-28 10:12:00,23,B00468,Registered,'97217,1976.0,Male
1,2,220,2011-07-28 10:21:00,23,2011-07-28 10:25:00,23,B00554,Registered,'02215,1966.0,Male
2,3,56,2011-07-28 10:33:00,23,2011-07-28 10:34:00,23,B00456,Registered,'02108,1943.0,Male
3,4,64,2011-07-28 10:35:00,23,2011-07-28 10:36:00,23,B00554,Registered,'02116,1981.0,Female
4,5,12,2011-07-28 10:37:00,23,2011-07-28 10:37:00,23,B00554,Registered,'97214,1983.0,Female


In [18]:
query = '''
SELECT s.* ,s1.station as "end station name" ,t.*
FROM trips t
INNER JOIN stations s ON t.start_station = s.id
INNER JOIN stations s1 ON t.end_station = s1.id

and s.id = 23
--GROUP BY s.station
ORDER BY 1 DESC
--LIMIT 5;
'''

run_query(query)

,id,station,municipality,lat,lng,end station name,id,duration,start_date,start_station,end_date,end_station,bike_number,sub_type,zip_code,birth_date,gender
0,23,Mayor Thomas M. Menino - Government Center,Boston,42.359677000000005,-71.059364,Mayor Thomas M. Menino - Government Center,1,9,2011-07-28 10:12:00,23,2011-07-28 10:12:00,23,B00468,Registered,'97217,1976,Male
1,23,Mayor Thomas M. Menino - Government Center,Boston,42.359677000000005,-71.059364,Mayor Thomas M. Menino - Government Center,2,220,2011-07-28 10:21:00,23,2011-07-28 10:25:00,23,B00554,Registered,'02215,1966,Male
2,23,Mayor Thomas M. Menino - Government Center,Boston,42.359677000000005,-71.059364,Mayor Thomas M. Menino - Government Center,3,56,2011-07-28 10:33:00,23,2011-07-28 10:34:00,23,B00456,Registered,'02108,1943,Male
3,23,Mayor Thomas M. Menino - Government Center,Boston,42.359677000000005,-71.059364,Mayor Thomas M. Menino - Government Center,4,64,2011-07-28 10:35:00,23,2011-07-28 10:36:00,23,B00554,Registered,'02116,1981,Female
4,23,Mayor Thomas M. Menino - Government Center,Boston,42.359677000000005,-71.059364,Mayor Thomas M. Menino - Government Center,5,12,2011-07-28 10:37:00,23,2011-07-28 10:37:00,23,B00554,Registered,'97214,1983,Female
5,23,Mayor Thomas M. Menino - Government Center,Boston,42.359677000000005,-71.059364,Mayor Thomas M. Menino - Government Center,6,19,2011-07-28 10:39:00,23,2011-07-28 10:39:00,23,B00456,Registered,'02021,1951,Male
6,23,Mayor Thomas M. Menino - Government Center,Boston,42.359677000000005,-71.059364,Mayor Thomas M. Menino - Government Center,7,24,2011-07-28 10:47:00,23,2011-07-28 10:47:00,23,B00554,Registered,'02140,1971,Female
7,23,Mayor Thomas M. Menino - Government Center,Boston,42.359677000000005,-71.059364,Mayor Thomas M. Menino - Government Center,8,7,2011-07-28 10:48:00,23,2011-07-28 10:48:00,23,B00554,Registered,'02140,1971,Female
8,23,Mayor Thomas M. Menino - Government Center,Boston,42.359677000000005,-71.059364,Mayor Thomas M. Menino - Government Center,9,8,2011-07-28 11:01:00,23,2011-07-28 11:01:00,23,B00554,Registered,'97214,1983,Female
9,23,Mayor Thomas M. Menino - Government Center,Boston,42.359677000000005,-71.059364,Mayor Thomas M. Menino - Government Center,64,15,2011-07-28 12:32:00,23,2011-07-28 12:32:00,23,B00062,Registered,'97214,1983,Female


# Create new DB & table

In [19]:
#intentional failure; SQL is syntactically correct
query = '''
CREATE TABLE IF NOT EXISTS MyTable(rid integer);
'''

#run_query(query)

In [20]:
create_table_sql = '''
CREATE TABLE IF NOT EXISTS MyTable(rid integer);
'''

cn2 = sqlite3.connect("MySqlite.DB")

c = cn2.cursor()

c.execute(create_table_sql)

MyQuery = """
SELECT * 
FROM sqlite_master
ORDER BY name;
"""

c.execute(MyQuery)

c.close()

pd.read_sql_query(MyQuery,cn2)

,type,name,tbl_name,rootpage,sql
0,table,MyTable,MyTable,2,CREATE TABLE MyTable(rid integer)


# INSERT, UPDATE, DELETE

In [21]:
MyInsert = 'insert into MyTable values(-199) ,(17);'

c = cn2.cursor()

c.execute(MyInsert)

cn2.commit()

c.close()

pd.read_sql_query("select * from MyTable;",cn2)


,rid
0,17
1,17
2,17
3,-199
4,17


In [22]:
MyUpdate = 'update MyTable set rid = 0 where rid = -199;'

c = cn2.cursor()

c.execute(MyUpdate)

cn2.commit()

c.close()

pd.read_sql_query("select * from MyTable;",cn2)


,rid
0,17
1,17
2,17
3,0
4,17


In [23]:
MyDelete = 'delete from MyTable where rid = 0;'

c = cn2.cursor()

c.execute(MyDelete)

cn2.commit()

c.close()

pd.read_sql_query("select * from MyTable;",cn2)

,rid
0,17
1,17
2,17
3,17
